# Representación del conocimiento

El proyecto se realizó con Python 3.9, utilizando solo las librerías estándar para el desarrollo. La ontología ejemplo que se utilizó para mostrar el funcionamiento de los predicados es la mencionada en [Pineda (2020)](https://www.sciencedirect.com/science/article/abs/pii/S0306457319309240) (Fig. 2. Non-Monotonic Ontology), a la cual únicamente se le hicieron cambios menores. El árbol de la ontología se muestra en la siguiente figura.

![](arbolinicio.png)

Importar una ontología. 

In [1]:
import json
with open('ontology.json') as json_file:
    data = json.load(json_file)
ontology = data['ontology']

In [2]:
#Guarda la ontología en un archivo 
def saveOntology(ontology):
    outData = {"ontology":ontology}
    with open('ontology.json', 'w') as outfile:
        json.dump(outData, outfile)

Importar funciones auxiliares:

In [3]:
from OntologyFunctions import *

## 1. Predicados para consultar

### a) Extensión de una clase

Obtiene el conjunto de todos los objetos que pertenecen a una clase nameClase (string) o subclase de la misma, dentro de la ontologia (list), ya sea porque se declaren directamente o porque están en la cerradura de la relación de herencia. 

In [4]:
def classExtention(nameClase,ontology):
    if not isInClass(nameClase,ontology): #Si no está en 
        return []                         #devolver una lista vacía
    individuals = []
    subClases = findSubClases(nameClase,ontology) #Obtener las sublclases de la clase
    indx = getIndex(nameClase, ontology) #Obtener el indice de la clase
    subClases.append(ontology[indx]) #Agregar la clase a la lista 
    for subClase in subClases: #Por cada clase o subclase
        for individual in subClase['individuals']: #Iterar por sus individuos
            individuals.append(individual['id'])   #Agregarlos a la lista de retorno
    return individuals

### Ejemplo 1

In [5]:
classExtention("top",ontology)

['pete', 'anne', 'mary', 'tom', '200', '201', '202', '203', 'prog', 'ai']

### Ejemplo 2

In [6]:
classExtention("abstract",ontology)

['prog', 'ai']

### b) La extensión de una propiedad 

Obtiene la extensión de una propiedad, muestra todos los objetos/clases que tienen una propiedad especifica, por declaración directa o por relación de herencia, incluyendo su valor respectivo. Para la implementación de este predicado se utiliza la función _getAllPropertiesThatMatch la cual encuentra todas las propiedades que coinciden con la ingresada por el usuario.

In [7]:
def propertyExtension(nameProperty, ontology, *, boolean=True):
    # Obtiene la raiz de la ontologia
    rootClass = superClassOfOntology(ontology)
    # Obtiene todos los individuos de la ontologia
    allIndividuals = classExtention(rootClass,ontology)
    # Obtiene todas las clases de la ontologia
    allClasses = [clase[1] for clase in hierarchyTree(ontology)]
    # Obtiene todos los entidades (individuos-clases) que cumplen con cierta propiedad
    valueIndividuals = getAllPropertiesThatMatch(nameProperty, allIndividuals, ontology, propertiesOf=propertiesOfObject,boolean=boolean)
    valueClasses = getAllPropertiesThatMatch(nameProperty, allClasses, ontology, propertiesOf=propertiesOfClass,boolean=boolean)
    # Une las dos entidades
    return valueIndividuals | valueClasses

### Ejemplo 1

In [8]:
 propertyExtension("study", ontology)

{'pete': 'uk', 'anne': 'mexico'}

### Ejemplo 2

In [9]:
 propertyExtension("sport", ontology)

{'pete': True, 'anne': True, 'tom': True, 'students': True}

### Ejemplo 3

In [10]:
 propertyExtension("sport", ontology, boolean=False)

{'mary': False,
 'people': False,
 'faculty members': False,
 'library staff': False}

### c) La extensión de una relación

Obtiene todos los individuos que tienen una relación ya sea por declaración directa o porque su clase o clases superiores tienen declarada esta extensión, muestra las extensiones y su valor.
 

In [11]:
def relation_extension(relacion,ontology):
    #clases que tienen la relacion de forma directa
    clases_ = clasesConEstaRelacion(relacion,ontology)
    
    #arbol de gerarquia para ordenar
    tree = hierarchyTree(ontology)
    tree.reverse()
    #ordenar las clases por aparicion gerarquica en la lista
    #clases mas al fondo del arbol con menos gerarquia
    clasesOrdenadas = []
    for class_tree in tree:
        for class_ in clases_:
            if class_ == class_tree[1]:
                clasesOrdenadas.append(class_)
    
    #buscar las subclases de cada clase que tiene la propiedad
    #para extenderle la relacion si no la tiene
    class_subclass = []
   
    for class_ in clasesOrdenadas:
        reg =[]
        reg.append([class_,[]])
        temp = []
        sub = findSubClases(class_,ontology)
        for sub_ in sub:
            temp.append(sub_["clase"])
        reg[0][1].extend(temp)
        class_subclass.extend(reg)
    
    ontologyCopy = copy.deepcopy(ontology)
    
    #heredar la relacion a subclases
    
    for class_root in class_subclass: ## por cada super clase aplica a cada una de sus subclases
        for sub_class in class_root[1]:#por cada subclase  de la superclase
            #verifica que la subclase tenga la relacion
            agregar = True
            # por cada relacion que contiene la sub clase, busca si tiene la relacion buscada
            for relacion_ in ontologyCopy[ getIndex(sub_class,ontologyCopy) ]["relations"]:
                #si la relacion existe marca agregar como falso, si no existe se agregara 
                if relacion_["subject"] == relacion:
                    agregar = False
                    break
            #si la subclase no tiene la relacion, heradala de su superclase
            if agregar is True:        
                # guarda la relcion que se debe de heredar
                rel_herencia = ontologyCopy[ getIndex(class_root[0],ontologyCopy) ]["relations"][getIndexOfRelation(relacion,class_root[0],ontologyCopy)]
                # se agrega la relacion a la subclase
                ontologyCopy[ getIndex(sub_class,ontologyCopy) ]["relations"].append(rel_herencia)
    clasesConRelacion = clasesConEstaRelacion(relacion,ontologyCopy)            
    # hasta aquí regresa las clases que tiene esa propiedad pero no su valor
    #return clasesConRelacion
    #print(clasesConRelacion)
    claseYvalorDeRelacion = []
    for class_ in clasesConRelacion:
        claseYvalorDeRelacion.append ([class_, ontologyCopy[ getIndex(class_,ontologyCopy) ]["relations"][getIndexOfRelation(relacion,class_,ontologyCopy)] ] )
    # hasta aquí existen las clases con su valor
    #return claseYvalorDeRelacion
    #print(claseYvalorDeRelacion)

    #para cada clase, ahora extender la herencia de la relacion a los sujetos de cada clase si es que no la tienen
    for class_ in clasesConRelacion:
        # obtiene la relacion de clase
        relacionDeClase = ontologyCopy[ getIndex(class_,ontologyCopy) ]["relations"][getIndexOfRelation(relacion,class_,ontologyCopy)]
        # obtiene la clase
        claseActual = ontologyCopy[ getIndex(class_,ontologyCopy) ]
        #indice temporal del individuo
        indiceIndividuo = -1
        
        #para cada individuo de la clase heredar la relacion si no la tiene declarada
        for i,individuo in enumerate(claseActual["individuals"]):
            #print(claseActual)
            #print(individuo["id"])
            # indica si es necesario que se agregue la relacion al individuo
            agregarRelacionAIndividuo = True
            for k,relacionesDeIndividuo in enumerate(individuo["relations"]):
                if relacionesDeIndividuo["subject"] == relacion: # ya esta la relacion en el individuo
                    agregarRelacionAIndividuo = False
                    break
                    
            if agregarRelacionAIndividuo is True:
                ontologyCopy[ getIndex(class_,ontologyCopy) ]["individuals"][i]["relations"].append(relacionDeClase)
    
    result = individualWithRelationship(relacion,ontologyCopy)
    return result
    

### Ejemplo 1

In [12]:
relation_extension("enrolled", ontology)

[['pete',
  {'type': 'value',
   'subject': 'enrolled',
   'object': 'prog',
   'value': False,
   'priority': 0}],
 ['anne',
  {'type': 'value',
   'subject': 'enrolled',
   'object': ['ai', 'prog'],
   'value': True,
   'priority': 0}]]

### Ejemplo 2

In [13]:
relation_extension("at", ontology)

[['prog',
  {'type': 'value', 'subject': 'at', 'object': ['200', '201'], 'priority': 0}],
 ['ai',
  {'type': 'value', 'subject': 'at', 'object': ['201', '202'], 'priority': 0}]]

### d) Todas las clases a las que pertenece un objeto

Obtiene una lista con todas las clases de la ontología (list)  a las que pertenece un objeto nameObject (string).

In [14]:
def allObjectClasses(nameObject, ontology):
    clases = []
    for clase in ontology: #Por cada clase de la ontología
         #Si el objeto pertenece a la extensión de la clase
        if nameObject in classExtention(clase['clase'],ontology):
            clases.append(clase['clase']) #El objeto pertenece a la clase
    return clases

### Ejemplo 1

In [15]:
allObjectClasses("pete", ontology)

['top', 'objects', 'concrete', 'people', 'students']

### Ejemplo 2

In [16]:
allObjectClasses("ai", ontology)

['top', 'objects', 'abstract', 'courses']

### e) Todas las propiedades de un objeto o clase

#### Propiedades de un objeto

Obtiene una lista con todas las propiedades de un objeto, ya sea por declaración directa o por relación de herencia. Regresa la descripción completa de cada una de las propiedades (valor, tipo, etc). Si no encuentra el objeto dentro de la ontología regresa un a lista vacía. Utiliza la función _getPropertiesByInheritance la cual retorna las propiedades relacionadas por herencia a ese objeto.


In [17]:
def propertiesOfObject(nameObject, ontology):
    
    # Obtiene todas las clases de la ontología (list) a las que pertenece un objeto nameObject (string)
    classes = allObjectClasses(nameObject, ontology)
    # Si no pertenece a ninguna clase mandar mensaje de error
    if not classes: 
        return []
    
    rootClass = superClassOfOntology(ontology)
    
    # Ordena las clases segun el grado de herencia
    classes = [clase['clase'] for clase in findSubClases(rootClass, ontology) 
              if clase['clase'] in classes]
    classes = classes[::-1]
    
    # Clase directa del individuo
    clase = getObjectClass(nameObject,ontology) 
    # Indice de la clase
    indxClass = getIndex(clase, ontology)
    # Indice del individuo en la clase
    indxObject = getIndexObject(nameObject, clase, ontology)
    
    # Obtiene las propiedades del individuo
    indvProp = ontology[indxClass]['individuals'][indxObject]['properties'].copy()
    
    # Obtiene los nombres de las propiedades elegidas hasta ese momento
    choosen = [element['name'] for element in indvProp]
    
    properties = getPropertiesByInheritance(classes, ontology, properties=indvProp, choosen=choosen)
    
    return properties

### Ejemplo 1

In [18]:
propertiesOfObject("pete", ontology)

[{'type': 'bool', 'name': 'teach', 'value': True, 'priority': 0},
 {'type': 'value', 'name': 'work', 'value': 'uk', 'bool': True, 'priority': 0},
 {'type': 'value',
  'name': 'born',
  'value': 'mexico',
  'bool': True,
  'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': False, 'priority': 0},
 {'type': 'value',
  'name': 'study',
  'value': 'uk',
  'bool': True,
  'priority': 0}]

### Ejemplo 2

In [19]:
propertiesOfObject("mary", ontology)

[{'type': 'bool', 'name': 'teach', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': False, 'priority': 0}]

#### Propiedades de una clase

Obtiene una lista con todas las propiedades de una clase, ya sean directas o por relación de herencia. Al igual que en las propiedades de objetos, la función retorna una descripción completa de las propiedades y de nuevo utiliza la función _getPropertiesByInheritance para obtener las propiedades por relación de herencia.

In [20]:
def propertiesOfClass(nameClass, ontology):
    
    # Obtiene la raiz de la ontologia
    rootClass = superClassOfOntology(ontology)
    # Si la clase raiz es la misma que nameClass
    # returna las propiedades solo de la clase raiz
    if nameClass == rootClass:
        # Obtiene las propiedades de la clase raiz
        classProp, _ = _directPropertiesOfClass(nameClass, ontology)
        # Regresa las propiedades de la clase
        return classProp
    
    # Se obtienen todas la jerarquia de clases a partir de una clase en especifico
    hierarchyClasses = obtenerSupClasesDe(nameClass, ontology)
    # Si la clase no encuentra ninguna superclase entonces la clase 
    # no esta en la ontologia
    if not hierarchyClasses: 
        return []
    
    # Agrgamos la clase a la jerarquia en el indice 0
    hierarchyClasses.insert(0, nameClass)
    
    properties = getPropertiesByInheritance(hierarchyClasses, ontology, imp=True)
    
    return properties

### Ejemplo 1

In [21]:
propertiesOfClass("students", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': False, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'work',
  'consequent': 'study',
  'value': [True, True],
  'priority': 3},
 {'type': 'imp',
  'antecedent': 'like',
  'consequent': 'study',
  'value': [True, True],
  'priority': 4},
 {'type': 'imp',
  'antecedent': 'born',
  'consequent': 'study',
  'value': [True, True],
  'priority': 5}]

### Ejemplo 2

In [22]:
propertiesOfClass("faculty members", ontology)

[{'type': 'bool', 'name': 'teach', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': False, 'priority': 0}]

### f) Todas las relaciones de un objeto o clase

#### Relaciones de un objeto

todas las relaciones de un objeto

In [23]:
def allRelationshipsOneObject(objeto,ontology):
    #obtener la clase a la cual pertenece el objeto
    clase = getObjectClass(objeto,ontology)
    #obtener todas las relaciones que tiene directamente o por herencia la clase
    relacionesDeClase = allRelationshipsOneClass(clase,ontology)
    #lista para guardar las relaciones que tiene el objeto localmente
    relacionesDeObjeto = []
    #obtener indice del individuo en su clase
    individuo = getIndexObject(objeto,clase,ontology)
    #obtener las relaciones directas del objeto
    for relacion_ in ontology[getIndex(clase,ontology)]["individuals"][individuo]["relations"]:
        relacionesDeObjeto.append(relacion_)
    
    #verificar de la lista de relacionde herencia por superclase
    #las que no tenga agregarlas
    for rel_class in relacionesDeClase: # para cada relacion de su superClases
        agregar = True
        for rel_ob in relacionesDeObjeto: # busca si esta en las relaciones de objeto si no agregasela
            if rel_class["subject"] == rel_ob["subject"]:
                agregar = False
                break
        
        if agregar is True:
            relacionesDeObjeto.append(rel_class)
    return relacionesDeObjeto

### Ejemplo 1

In [24]:
allRelationshipsOneObject("pete", ontology)

[{'type': 'value',
  'subject': 'enrolled',
  'object': 'prog',
  'value': False,
  'priority': 0}]

### Ejemplo 2

In [25]:
allRelationshipsOneObject("ai", ontology)

[{'type': 'value', 'subject': 'at', 'object': ['201', '202'], 'priority': 0}]

#### Relaciones de una clase

todas las relaciones de una clase

In [26]:
def allRelationshipsOneClass(clase,ontology):
    #optener superClases de la clase
    superClasesDeClase = obtenerSupClasesDe(clase,ontology)
    #optener la clase que se esta buscando
    clase_actual = ontology[ getIndex(clase,ontology) ]
    #almacen de todas las relaciones de esta clase por declaracion directa en la misma
    relacionesDeClase = []
    
    for relInClass in clase_actual["relations"]:
        relacionesDeClase.append(relInClass)
        
    #para cada super clase en forma ascendente, busca
    for claseSup in superClasesDeClase:
        #obtener la clase de la ontologia
        superClase = ontology[ getIndex(claseSup,ontology) ]
        #para cada relacion en la superClase, busca si debo agregar alguna a la subclase
        for relInSupClass in superClase["relations"]:
            #busca en las relaciones que ya tiene la clase
            guardar = True 
            for relClassActual in relacionesDeClase:
                if relClassActual["subject"] == relInSupClass["subject"]:
                    guardar = False
                    break
            
            if guardar is True:
                relacionesDeClase.append(relInSupClass)
    return relacionesDeClase

### Ejemplo 1

In [27]:
allRelationshipsOneClass("students", ontology)

[{'type': 'value',
  'subject': 'enrolled',
  'object': 'courses',
  'value': True,
  'priority': 0}]

### Ejemplo 2

In [28]:
allRelationshipsOneClass("faculty members", ontology)

[]

## 2. Predicados para añadir:

### A.1 Clases

Añade un objeto individualId (string) a una clase nameClass (string) de la ontología (list)

In [29]:
def addObject(individualId,nameClass,ontology):
    if not isInClass(nameClass,ontology): # Si no existe la clase en la ontología: 
        #raise Error('Error: La Clase no esta en la ontologia.') # Mandar mensaje de error
        return False
    individual = {'id':individualId, 'properties':[], 'relations':[]} #Crear diccionario de individuo
    indxClass = getIndex(nameClass, ontology) # Obtener el indice de la clase del nuevo individuo
    ontology[indxClass]['individuals'].append(individual) # Agregar el  nuevo individuo a la lista de individuos
    saveOntology(ontology) #Guardar la ontología

### Ejemplo 1

Extension de la clase "library staff" antes de agregar un nuevo individuo.

In [30]:
classExtention("library staff", ontology)

['tom']

Se agrega un nuevo individuo a la clase "library staff".

In [31]:
addObject("jose", "library staff", ontology)

Extension de la clase "library staff" despues de agregar un nuevo individuo.

In [32]:
classExtention("library staff", ontology)

['tom', 'jose']

### Ejemplo 2

Extension de la clase "people" antes de agregar un nuevo individuo.

In [33]:
classExtention("people", ontology)

['pete', 'anne', 'mary', 'tom', 'jose']

Se agrega un nuevo individuo a la clase "people".

In [34]:
addObject("roberto", "people", ontology)

Extension de la clase "people" despues de agregar un nuevo individuo.

In [35]:
classExtention("people", ontology)

['pete', 'anne', 'mary', 'tom', 'jose', 'roberto']

### A.2 Objetos

Añade una clase nameClass (string), especificando su clase madre nameSuperClass (string) en la  ontología (list)

In [36]:
def addClass(nameClass,nameSuperClass,ontology):
    if nameSuperClass == None: # Si se quiere agregar una clase raíz
        for clase in ontology: # Encontrar quien era la clase raíz
            if clase["mother"] == None:
                clase["mother"] = nameClass  #Cambiar su madre por la clase agregada
    else:
        if not isInClass(nameSuperClass,ontology): #Si no existe la clase en la ontología
            #raise Error('Error: La Clase no esta en la ontologia.') #Mandar mensaje de error
            return False
    #Crear diccionario para la clase
    newClass = {'clase':nameClass, 
                'mother':nameSuperClass,
                'properties':[], 
                'relations':[], 
                'individuals':[]}
    ontology.append(newClass) # Agregar la clase a la ontología 
    saveOntology(ontology) # Guardar la ontología

### Ejemplo 1

La jerarquía de clases original

In [37]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'rooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff']]

Se agrega una nueva clase a la ontologia "teachers" a la clase madre "people".

In [38]:
addClass("teachers","faculty members",ontology)

La jerarquía de clases despues de agregar la clase "teachers"

In [39]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'rooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [5, 'teachers']]

Super clases de la nueva clase "teachers".

In [40]:
obtenerSupClasesDe("teachers",ontology)

['faculty members', 'people', 'concrete', 'objects', 'top']

### Ejemplo 2

La jerarquía de clases original

In [41]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'rooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [5, 'teachers']]

Se agrega una nueva clase a la ontologia "restrooms" a la clase madre "rooms".

In [42]:
addClass("restrooms","rooms",ontology)

La jerarquía de clases despues de agregar la clase "restrooms"

In [43]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'rooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [4, 'restrooms'],
 [5, 'teachers']]

Super clases de la nueva clase "teachers".

In [44]:
obtenerSupClasesDe("restrooms",ontology)

['rooms', 'concrete', 'objects', 'top']

### B.1 Propiedades de clases 

Añadir a una clase nameClass (string), propiedades propertyName (string), con cierto valor
propertyValue (string), dentro de una ontologia (list). Si la propiedad necesita una
prioridad priority (int) se le puede agregar.

In [45]:
def addClassProperty(nameClass, propertyName, propertyValue, ontology, *, boolean=True, priority=None):
    if not isInClass(nameClass,ontology): #Si la clase no esta en la ontología
        return False
    indxClass = getIndex(nameClass, ontology) #Indice de la clase 
    if priority is not None: #Si hay prioridad la propiedad es de implicación
        newProperty = {'type':'imp',
                       'antecedent':propertyName[0],
                       'consequent':propertyValue[0],
                       'value': propertyName[1] + [propertyValue[1]],
                       'priority':priority}
    # De otra forma la propiedad puede ser boleana o de valor
    elif type(propertyName) is not list and type(propertyValue) is not list: 
        if type(propertyValue) is bool:
            newProperty = {'type':"bool", #Solo se diferencian por su tipo
                           'name':propertyName,
                           'value':propertyValue, 
                           'priority':0} #La prioridad por default es 0
        else:
            newProperty = {'type':"value", #Solo se diferencian por su tipo
                           'name':propertyName,
                           'value':propertyValue,
                           'bool': boolean,
                           'priority':0} #La prioridad por default es 0
    else:
        return False
        
    ontology[indxClass]['properties'].append(newProperty) #Agregar a la lista de propiedades de la clase
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Propiedades de la clase "teachers".

In [46]:
propertiesOfClass("teachers", ontology)

[{'type': 'bool', 'name': 'teach', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': False, 'priority': 0}]

Se agrega una propiedad boleana "teach" con el valor True.

In [47]:
addClassProperty("teachers", "teach", True, ontology)

Se agrega una propiedad de valor "sni" con el valor "nivel1" negada.

In [48]:
addClassProperty("teachers", "sni", "nivel1", ontology, boolean=False)

Se agrega una propiedad de implicacion con un solo antecedente positivo.

In [49]:
addClassProperty("teachers", [["sni"],[True]], ["researcher",True], ontology, priority=0)

Propiedades de la clase "teachers" despues de añadir nuevos elementos.

In [50]:
propertiesOfClass("teachers", ontology)

[{'type': 'bool', 'name': 'teach', 'value': True, 'priority': 0},
 {'type': 'value',
  'name': 'sni',
  'value': 'nivel1',
  'bool': False,
  'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'sni',
  'consequent': 'researcher',
  'value': [True, True],
  'priority': 0}]

### B.2 Propiedades de objetos


Añadir a un objeto nameObject (string) de la ontologia (list), una propiedad propertyName (string), con cierto valor propertyValue (bool, string)

In [51]:
def addObjectProperty(nameObject, propertyName, propertyValue, ontology, *, boolean=True):
    clase = getObjectClass(nameObject,ontology) #Clase directa del individuo
    if not clase: # Si no pertenece a ninguna clase mandar mensaje de error
        return False
    indxClass = getIndex(clase, ontology) #Indice de la clase
    
    #Nueva propiedad para el individuo: colocar nombre del atributo y su valor
    if type(propertyValue) is bool:
        newProperty = {'type':"bool", #Solo se diferencian por su tipo
                       'name':propertyName,
                       'value':propertyValue, 
                       'priority':0} #La prioridad por default es 0
    else:
        newProperty = {'type':"value", #Solo se diferencian por su tipo
                       'name':propertyName,
                       'value':propertyValue,
                       'bool': boolean,
                       'priority':0} #La prioridad por default es 0
        
    indxObject = getIndexObject(nameObject, clase, ontology) #indice del individuo en la clase
    ontology[indxClass]["individuals"][indxObject]["properties"].append(newProperty)#Agregar a la lista de propiedades del individuo
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Propiedades del objeto "roberto".

In [52]:
propertiesOfObject("roberto", ontology)

[{'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': False, 'priority': 0}]

Se agrega una propiedad boleana "sport" con el valor True.

In [53]:
addObjectProperty("roberto", "sport", True, ontology)

Se agrega una propiedad de valor "size" con el valor "small" positiva.

In [54]:
addObjectProperty("roberto", "size", "small", ontology)

Propiedades del "objeto" despues de añadir nuevos elementos.

In [55]:
propertiesOfObject("roberto", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'value',
  'name': 'size',
  'value': 'small',
  'bool': True,
  'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0}]

### C.1 Relaciones de clases 

Agrega una relacion a una clase con otra clase u objeto

In [56]:
def addClassRelation(nameClass, newRelation, otherClasses_Objects, ontology, value=True):
    
    if not isInClass(nameClass, ontology):
        #raise Error('Error: La clase no esta en la ontologia.')
        return False
    
    if type(otherClasses_Objects) is not list:
        otherClasses_Objects = [otherClasses_Objects]
    
    for element in otherClasses_Objects:
        if not isInClass(element, ontology):
            if not getObjectClass(element,ontology):
                ##raise Error('Error: La clase o individuo para asignar esa relacion no esta en la ontologia.')
                return False
            
    objects = otherClasses_Objects[0] if len(otherClasses_Objects) == 1 else otherClasses_Objects
    
    newRelation = {'type':'value',
                   'subject': newRelation,
                   'object': objects,
                   'value': value,
                   'priority': 0}
    
    # Indice de la clase
    indxClass = getIndex(nameClass, ontology)
    
    ontology[indxClass]['relations'].append(newRelation)
    saveOntology(ontology)

### Ejemplo 1

Relaciones de la clase "teachers".

In [57]:
allRelationshipsOneClass("teachers", ontology)

[]

Se agrega una relacion positiva "lectures" relacionadola con la clase "courses".

In [58]:
addClassRelation("teachers", "lectures", "courses", ontology)

Se agrega una relacion negativa "friends" relacionadola con el objeto "tom".

In [59]:
addClassRelation("teachers", "friends", "tom", ontology, value=False)

Relaciones de la clase "teachers" despues de las modificaciones.

In [60]:
allRelationshipsOneClass("teachers", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': 'courses',
  'value': True,
  'priority': 0},
 {'type': 'value',
  'subject': 'friends',
  'object': 'tom',
  'value': False,
  'priority': 0}]

### C.2 Relaciones de clases 

Agrega una relacion a un objeto con otra clase u objeto

In [61]:
def addObjectRelation(nameObject, newRelation, otherClasses_Objects, ontology, value=True):
    
    # Clase directa del individuo
    clase = getObjectClass(nameObject,ontology) 
    # Si no pertenece a ninguna clase mandar mensaje de error
    if not clase: 
        #raise Error('Error: El individuo no esta en la ontologia.')
        return False
    
    if type(otherClasses_Objects) is not list:
        otherClasses_Objects = [otherClasses_Objects]
    
    for element in otherClasses_Objects:
        if not isInClass(element, ontology):
            if not getObjectClass(element,ontology):
                #raise Error('Error: La clase o individuo para asignar esa relacion no esta en la ontologia.')
                return False
            
    objects = otherClasses_Objects[0] if len(otherClasses_Objects) == 1 else otherClasses_Objects
    
    newRelation = {'type':'value',
                   'subject': newRelation,
                   'object': objects,
                   'value': value,
                   'priority': 0}
    
    # Indice de la clase
    indxClass = getIndex(clase, ontology)
    # Indice del individuo en la clase
    indxObject = getIndexObject(nameObject, clase, ontology)
    
    ontology[indxClass]['individuals'][indxObject]['relations'].append(newRelation)
    saveOntology(ontology)

### Ejemplo 1

Relaciones del objeto "jose".

In [62]:
allRelationshipsOneObject("jose", ontology)

[]

Se agrega una relacion positiva "lover" relacionadola con el objeto "mary".

In [63]:
addObjectRelation("jose", "lover", "mary", ontology, value=True)

Se agrega una relacion positiva "cleans" relacionadola con la clase "rooms".

In [64]:
addObjectRelation("jose", "cleans", "rooms", ontology, value=True)

Relaciones del objeto "jose" despues de las modificaciones.

In [65]:
allRelationshipsOneObject("jose", ontology)

[{'type': 'value',
  'subject': 'lover',
  'object': 'mary',
  'value': True,
  'priority': 0},
 {'type': 'value',
  'subject': 'cleans',
  'object': 'rooms',
  'value': True,
  'priority': 0}]

## 3. Predicados para eliminar

### A.1 Clases

Elimina la clase nameClass (string) y las subclases de la misma de una ontología (list), junto con sus individuos.

In [66]:
def removeClass(nameClass, ontology):
    
    indxClass = getIndex(nameClass, ontology) #Indice de la clase 
    if not isInClass(nameClass,ontology): # Si no existe la clase en la ontología: 
        #raise Error('Error: La Clase no esta en la ontologia.') # Mandar mensaje de error
        return False
    ind = ontology[indxClass]['individuals']
    mother = ontology[indxClass]['mother']
    ontology.pop(indxClass) #Eliminar la clase de la ontologia
    for clase in ontology:
        if clase['mother'] == nameClass:
            clase['mother'] = mother
    indxMother = getIndex(mother, ontology) #Indice de la clase 
    ontology[indxMother]['individuals'].extend(ind)
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Arbol jerarquico de la ontologia antes de eliminar una clase.

In [67]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'rooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [4, 'restrooms'],
 [5, 'teachers']]

Se elimina la clase "rooms".

In [68]:
removeClass("rooms",ontology)

Arbol jerarquico de la ontologia despues de eliminar una clase.

In [69]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'restrooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [5, 'teachers']]

### A.2 Objetos

Elimina un objeto nameObject (string) de una ontología (list)

In [70]:
def removeObject(nameObject, ontology):
    nameClass = getObjectClass(nameObject,ontology) #Clase del objeto
    if not nameClass:
        #raise Error('Error: El objeto no existe.')
        return False
    indexClass = getIndex(nameClass, ontology) #Indice de la clase
    indexObject = getIndexObject(nameObject, nameClass, ontology) #Indice del objeto
    ontology[indexClass]["individuals"].pop(indexObject) #Eliminar objeto 
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Extension de la clase "people".

In [71]:
classExtention("people", ontology)

['pete', 'anne', 'mary', 'tom', 'jose', 'roberto']

Se elimina el objeto "roberto".

In [72]:
removeObject("roberto", ontology)

Extension de la clase "people" despues de eliminar el objeto.

In [73]:
classExtention("people", ontology)

['pete', 'anne', 'mary', 'tom', 'jose']

### B.1 Propiedades específicas de clases

Elimina propiedades específicas de una clase nameClass (string) de una ontología (list):
Si la propiedad es de tipo bool o value: 
- propertyClass (string), consequent = None.
- Si la propiedad es de implicacion: 
- propertyClass (list: atributo, valor), consequent (list: atributo, valor).

In [74]:
def removeClassProperty(propertyClass, nameClass, ontology, consequent = None):
    indxProp = getPropertyClassIndex(propertyClass, nameClass, ontology, consequent) #indice de la propiedad
    indxClass = getIndex(nameClass, ontology) #indice de la clase
    ontology[indxClass]['properties'].pop(indxProp) #Eliminar propiedad
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Se obtienen las propiedades de la clase "students".

In [75]:
propertiesOfClass("students", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': False, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'work',
  'consequent': 'study',
  'value': [True, True],
  'priority': 3},
 {'type': 'imp',
  'antecedent': 'like',
  'consequent': 'study',
  'value': [True, True],
  'priority': 4},
 {'type': 'imp',
  'antecedent': 'born',
  'consequent': 'study',
  'value': [True, True],
  'priority': 5}]

Se elimina la propiedad de implicacion "born( ) =>> study( )".

In [76]:
removeClassProperty([["born"],[True]], "students", ontology, consequent = ["study",True])

Se obtienen las propiedades de la clase "students" despues de eliminar la propiedad.

In [77]:
propertiesOfClass("students", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': False, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'work',
  'consequent': 'study',
  'value': [True, True],
  'priority': 3},
 {'type': 'imp',
  'antecedent': 'like',
  'consequent': 'study',
  'value': [True, True],
  'priority': 4}]

### B.2 Propiedades específicas de Objeto

Elimina propiedades especificas propertyObject (String) de un objeto nameObject (string), pueden ser booleanas o de valor.

In [78]:
def removeObjectProperty(propertyObject, nameObject, ontology):
    indxProperty = getPropertyObjectIndex(propertyObject, nameObject, ontology)
    #Falta poner caso en donde no exista la propiedad en el objeto
    nameClass = getObjectClass(nameObject, ontology) #Clase del objeto
    indxClass = getIndex(nameClass, ontology) # indice de clase en la ontologia
    indxObject = getIndexObject(nameObject, nameClass, ontology) #Indice del objeto en la clase
    ontology[indxClass]['individuals'][indxObject]['properties'].pop(indxProperty) #eliminar propiedad
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Propiedades del objeto "tom".

In [79]:
propertiesOfObject("tom", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'value',
  'name': 'size',
  'value': 'short',
  'bool': True,
  'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0}]

Se elimina la propiedad "size" del objeto "tom".

In [80]:
removeObjectProperty("size", "tom", ontology)

Propiedades actualizadas del objeto "tom".

In [81]:
propertiesOfObject("tom", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0}]

### C.1 Relaciones específicas de clases

Eliminar relaciones específicas de tipo value de clases 

In [82]:
def removeClassRelation(subjectRelation, objectRelation, nameClass, ontology):
    indxCR = getRelationClassIndex(subjectRelation, objectRelation, nameClass, ontology) #indice de la relacion en clase
    # if not indxCR:
    #     raise Error('No existe relacion en la clase')
    indxClass = getIndex(nameClass,ontology) #indice de la clase en la ontologia
    ontology[indxClass]['relations'].pop(indxCR) #Eliminar relacion
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Se obtienen todas las relaciones de la clase "teachers".

In [83]:
allRelationshipsOneClass("teachers", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': 'courses',
  'value': True,
  'priority': 0},
 {'type': 'value',
  'subject': 'friends',
  'object': 'tom',
  'value': False,
  'priority': 0}]

Se elimina la relacion "friends" de la clase "teachers".

In [84]:
removeClassRelation("friends", "tom", "teachers", ontology)

Se obtienen las relaciones actualizadas de la clase "teachers".

In [85]:
allRelationshipsOneClass("teachers", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': 'courses',
  'value': True,
  'priority': 0}]

### C.2 Relaciones específicas de objetos

Eliminar relaciones específicas de objetos

In [86]:
def removeObjectRelation(subjectRelation, objectRelation, nameObject, ontology):
    indxRO = getRelationObjectIndex(subjectRelation, objectRelation, nameObject, ontology)
    nameClass = getObjectClass(nameObject, ontology)
    indxClass = getIndex(nameClass, ontology) #Indice de clase en ontologia 
    indxObj = getIndexObject( nameObject, nameClass, ontology) # indice de objeto en clase
    ontology[indxClass]['individuals'][indxObj]['relations'].pop(indxRO) #Eliminar relacion de objeto
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Se obtienen las relaciones del objeto "jose".

In [87]:
allRelationshipsOneObject("jose", ontology)

[{'type': 'value',
  'subject': 'lover',
  'object': 'mary',
  'value': True,
  'priority': 0},
 {'type': 'value',
  'subject': 'cleans',
  'object': 'rooms',
  'value': True,
  'priority': 0}]

Se elimina la relacion "lover" del objeto "jose".

In [88]:
removeObjectRelation("lover", "mary", "jose", ontology)

Se obtienen las relaciones actualizadas del objeto "jose".

In [89]:
allRelationshipsOneObject("jose", ontology)

[{'type': 'value',
  'subject': 'cleans',
  'object': 'rooms',
  'value': True,
  'priority': 0}]

## 4. Predicados para modificar

### A.1 El nombre de una clase

Modifica el nombre de una clase nameClase (string), de una por otro nombre newNameClass (string).

In [90]:
def changeNameClass(nameClass,newNameClass,ontology):
    indexClass = getIndex(nameClass, ontology) #Indice de la clase
    if not isInClass(nameClass,ontology): # Si no existe la clase en la ontología: 
        #raise Error('Error: La Clase no esta en la ontologia.') # Mandar mensaje de error
        return False
    ontology[indexClass]["clase"] = newNameClass #Cambiar el nombre de la clase
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Se obtienen el arbol jerárquico con los nombres de clase.

In [91]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'restrooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [5, 'teachers']]

Se modifica el nombre de "teachers" por "profesors".

In [92]:
changeNameClass("teachers","professors",ontology)

Se obtiene el arbol jerárquico con los nombres actualizados.

In [93]:
hierarchyTree(ontology)

[[0, 'top'],
 [1, 'objects'],
 [2, 'concrete'],
 [2, 'abstract'],
 [3, 'people'],
 [3, 'restrooms'],
 [3, 'courses'],
 [4, 'students'],
 [4, 'faculty members'],
 [4, 'library staff'],
 [5, 'professors']]

### A.2 El nombre de un objeto

Modifica el nombre de una objeto nameObject (string), por otro newNameObject (string).

In [94]:
def changeNameObject(nameObject,newNameObject,ontology):
    nameClass = getObjectClass(nameObject,ontology) #Clase del objeto
    if not nameClass: # Si no existe la clase en la ontolgia
        #raise Error('Error: El objeto no está en la ontología.')
        return False
    indexClass = getIndex(nameClass, ontology) #Indice de la clase
    indexObject = getIndexObject(nameObject, nameClass, ontology) #Indice del objeto
    ontology[indexClass]["individuals"][indexObject]["id"] = newNameObject #Cambiar el nombre del objeto
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Se obtiene la extension de la clase "library staff".

In [95]:
classExtention("library staff", ontology)

['tom', 'jose']

El objeto "jose" cambia de nombre a "roberto".

In [96]:
changeNameObject("jose","roberto",ontology)

Se obtiene la extension de la clase "library staff" actualizada.

In [97]:
classExtention("library staff", ontology)

['tom', 'roberto']

### B.1 El valor de una propiedad específica de una clase

Cambia el valor dentro de una propiedad especifica de una clase.

In [98]:
def change_value_class_property(nameClass,propertyName,propertyValue,ontology,*,boolean = True, priority = None):
    if not isInClass(nameClass,ontology): #Si la clase no esta en la ontología
        #raise Error('Error: La Clase no esta en la ontologia.') #Mandar mensaje de error
        return False
    indxClass = getIndex(nameClass, ontology) #Indice de la clase 
    #Si hay prioridad la propiedad es de implicación
    if priority is not None and type(propertyName) is list and type(propertyValue) is list: 
        newProperty = {'type':'imp',
                       'antecedent':propertyName[0],
                       'consequent':propertyValue[0],
                       'value': propertyName[1] + [propertyValue[1]],
                       'priority':priority}
    # De otra forma la propiedad puede ser boleana o de valor
    elif type(propertyName) is not list and type(propertyValue) is not list: 
        if type(propertyValue) is bool:
            newProperty = {'type':"bool", #Solo se diferencian por su tipo
                           'name':propertyName,
                           'value':propertyValue, 
                           'priority':0} #La prioridad por default es 0
        else:
            newProperty = {'type':"value", #Solo se diferencian por su tipo
                           'name':propertyName,
                           'value':propertyValue,
                           'bool': boolean,
                           'priority':0} #La prioridad por default es 0
    else:
        #raise Error('Error: Si es propiedad de implicacion agregar prioridad.')
        return False
        
    typePropertyNameField=""
    propertyName_=""
    if newProperty["type"] == "imp":
        if len(newProperty["antecedent"]) == 1:
            propertyName_= newProperty["antecedent"][0]
        else:
            propertyName_= newProperty["antecedent"]
    else:    
        propertyName_= newProperty["name"]
    
    #busca la propiedad en la clase
    confirmacion = False
    for i,property_ in enumerate(ontology[indxClass]['properties']):
        #identificar como se llama la llave con la que buscar el identificado de la propiedad
        if property_["type"] == "imp":     
            typePropertyNameField = "antecedent"
        else:
            typePropertyNameField = "name"
        #encuentra la propiedad actual con ese nombre
        if property_[typePropertyNameField] == propertyName_: 
            if property_["type"] == newProperty["type"]:  #si las estrcuturas de propiedad son las mismas
                ontology[indxClass]['properties'][i] = newProperty
                saveOntology(ontology) #Guardar ontología solo cuando hubo cambios
                confirmacion = True
            else:
                print("No son del mismo tipo")
                

    return confirmacion

### Ejemplo 1

Propiedades de la clase "students".

In [99]:
propertiesOfClass("students", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': False, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'work',
  'consequent': 'study',
  'value': [True, True],
  'priority': 3},
 {'type': 'imp',
  'antecedent': 'like',
  'consequent': 'study',
  'value': [True, True],
  'priority': 4}]

Cambia el valor de la propiedad "fun" de negativa a positiva de la clase "students".

In [100]:
change_value_class_property("students","fun",True,ontology)

True

Cambia el consecuente de la propiedad de implicacion con antecedente "like".

In [101]:
change_value_class_property("students",[["like"],[True]],["work", True],ontology, priority=0)

True

Propiedades actualizadas de la clase "students".

In [102]:
propertiesOfClass("students", ontology)

[{'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0},
 {'type': 'imp',
  'antecedent': 'work',
  'consequent': 'study',
  'value': [True, True],
  'priority': 3},
 {'type': 'imp',
  'antecedent': 'like',
  'consequent': 'work',
  'value': [True, True],
  'priority': 0}]

### B.2 El valor de una propiedad específica de un objeto

Cambia el valor dentro de una propiedad especifica de un objeto.

In [103]:
def change_value_object_property(nameObject,propertyName,propertyValue,ontology,*,boolean=True):
    class_name = getObjectClass(nameObject,ontology)
    object_index = getIndexObject(nameObject,class_name,ontology)
    
    if type(propertyValue) is bool:
        newProperty = {'type':"bool", #Solo se diferencian por su tipo
                       'name':propertyName,
                       'value':propertyValue, 
                       'priority':0} #La prioridad por default es 0
    else:
        newProperty = {'type':"value", #Solo se diferencian por su tipo
                       'name':propertyName,
                       'value':propertyValue,
                       'bool': boolean,
                       'priority':0} #La prioridad por default es 0
        
    #busca la propiedad que va actualizar y aplica el cambio
    confirmacion = False
    for i,property_ in enumerate(ontology[getIndex(class_name,ontology)]["individuals"][object_index]["properties"]):
        if property_["name"] == propertyName :
            if property_["type"] == newProperty["type"]:
                ontology[getIndex(class_name,ontology)]["individuals"][object_index]["properties"][i] = newProperty
                confirmacion = True
                
            else:
                print("No son del mismo tipo")
    return confirmacion
    saveOntology(ontology) #Guardar ontología

### Ejemplo 1

Propiedades del objeto "anne".

In [104]:
propertiesOfObject("anne", ontology)

[{'type': 'value', 'name': 'work', 'value': 'us', 'bool': True, 'priority': 0},
 {'type': 'value',
  'name': 'study',
  'value': 'mexico',
  'bool': True,
  'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0}]

Se cambia el valor de la propiedad study del objeto "anne".

In [105]:
change_value_object_property("anne","study","uk",ontology)

True

Propiedades del objeto "anne" actualizadas.

In [106]:
propertiesOfObject("anne", ontology)

[{'type': 'value', 'name': 'work', 'value': 'us', 'bool': True, 'priority': 0},
 {'type': 'value',
  'name': 'study',
  'value': 'uk',
  'bool': True,
  'priority': 0},
 {'type': 'bool', 'name': 'sport', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'fun', 'value': True, 'priority': 0},
 {'type': 'bool', 'name': 'teach', 'value': False, 'priority': 0}]

### C.1 Con quién mantiene una relación específica una clase

Cambia el objeto o clase con quien mantiene una relacion especifica una clase.

In [107]:
def changeClassRelation(nameClass, nameRelation, newClasses_Objects, ontology):
    
    if not isInClass(nameClass, ontology):
        ##raise Error('Error: La clase no esta en la ontologia.')
        return False
        
    if type(newClasses_Objects) is not list:
        newClasses_Objects = [newClasses_Objects]
    
    for element in newClasses_Objects:
        if not isInClass(element, ontology):
            if not getObjectClass(element,ontology):
                #raise Error('Error: La clase o individuo para asignar esa relacion no esta en la ontologia.')
                return False
        
    # Indice de la clase
    indxClass = getIndex(nameClass, ontology)
    
    for relation in ontology[indxClass]['relations']:
        if relation['subject'] == nameRelation:
            if len(newClasses_Objects) == 1:
                relation['object'] = newClasses_Objects[0]
            else:
                relation['object'] = newClasses_Objects
            saveOntology(ontology)
            return
    
    #raise Error('Error: La relacion no esta en la clase de la ontologia.')
    return False

### Ejemplo 1

Relaciones de la clase "profesors".

In [108]:
allRelationshipsOneClass("professors", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': 'courses',
  'value': True,
  'priority': 0}]

Cambia la clase con la que mantiene la relacion "lectures".

In [109]:
changeClassRelation("professors", "lectures", "abstract", ontology)

Relaciones actualizadas de la clase "profesors".

In [110]:
allRelationshipsOneClass("professors", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': 'abstract',
  'value': True,
  'priority': 0}]

### C.2 Con quién mantiene una relacióne específica un objeto

Cambiar con quien mantiene una relación específica un objeto

In [111]:
def changeObjectRelation(nameObject, nameRelation, newClasses_Objects, ontology):
    # Clase directa del individuo
    clase = getObjectClass(nameObject,ontology) 
    # Si no pertenece a ninguna clase mandar mensaje de error
    if not clase: 
        #raise Error('Error: El individuo no esta en la ontologia.')
        return False
    
    # Si el individuo para asignar la relacion no existe manda error
    if type(newClasses_Objects) is not list:
        newClasses_Objects = [newClasses_Objects]
        
    for element in newClasses_Objects:
        if not isInClass(element, ontology):
            if not getObjectClass(element,ontology):
                #raise Error('Error: La clase o individuo para asignar esa relacion no esta en la ontologia.')
                return False
            
    # Indice de la clase
    indxClass = getIndex(clase, ontology)
    # Indice del individuo en la clase
    indxObject = getIndexObject(nameObject, clase, ontology)
    
    for relation in ontology[indxClass]['individuals'][indxObject]['relations']:
        if relation['subject'] == nameRelation:
            if len(newClasses_Objects) == 1:
                relation['object'] = newClasses_Objects[0]
            else:
                relation['object'] = newClasses_Objects
            saveOntology(ontology) #Guardar ontología
            return 
        
    #raise Error('Error: La relacion no esta en el individuo de la ontologia.')
    return False

### Ejemplo 1

Relaciones del objeto "tom".

In [112]:
allRelationshipsOneObject("tom", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': ['prog'],
  'value': True,
  'priority': 0}]

Cambia con quien mantiene la relacion "lectures".

In [113]:
changeObjectRelation("tom", "lectures", ["prog", "ai"], ontology)

Relaciones actualizadas del objeto "tom".

In [114]:
allRelationshipsOneObject("tom", ontology)

[{'type': 'value',
  'subject': 'lectures',
  'object': ['prog', 'ai'],
  'value': True,
  'priority': 0}]

## Arbol Final

Por último el árbol de la ontología resultante se muestra en la siguiente figura, donde se pueden observar todos los cambios realizados por los predicados anteriores.

![](arbolFinal.png)